First, add the dataset that contains the pre-trained model and the installation for detectron2:
https://www.kaggle.com/tjungblut/nflimpactdetectron

Then you can proceed to install it via pip and offline installation:

In [1]:
!python3 -m pip install ../input/nflimpactdetectron/detectron2/detectron2 --no-index --find-links ../input/nflimpactdetectron/wheelz/wheelz

Looking in links: ../input/nflimpactdetectron/wheelz/wheelz
Processing /kaggle/input/nflimpactdetectron/detectron2/detectron2
Processing /kaggle/input/nflimpactdetectron/wheelz/wheelz/yacs-0.1.8-py3-none-any.whl
Processing /kaggle/input/nflimpactdetectron/wheelz/wheelz/fvcore-0.1.2-py3-none-any.whl
Processing /kaggle/input/nflimpactdetectron/wheelz/wheelz/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl
  Created wheel for detectron2: filename=detectron2-0.3-cp37-cp37m-linux_x86_64.whl size=5311693 sha256=f02d4486cfab3f392b29c70b2380e4563a36f4d282a28b627d18a5b628b29b4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fo7hjk5w/wheels/a8/a2/bd/c673fab154eab56fe2c82c66f310d2a33bc4a1aaefaa8182e2
Successfully built detectron2


In [2]:
# importing all needed libs:

import cv2
import numpy as np 
import pandas as pd
import os
import subprocess

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
plt.rcParams['figure.dpi'] = 150

import seaborn as sns

from IPython.display import Video, display
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.structures import Instances, BoxMode
from detectron2.utils.visualizer import Visualizer


In [3]:
# this basically creates the detectron2 configuration for our pre-trained model
def new_model_cfg():
    cfg = get_cfg()
    model = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"
    cfg.merge_from_file(model_zoo.get_config_file(model))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    cfg.MODEL.MASK_ON = False
    cfg.INPUT.RANDOM_FLIP = "none"
    cfg.OUTPUT_DIR = "output"
    cfg.DATASETS.TRAIN = ("nflimpact_train",)
    cfg.DATASETS.TEST = ("nflimpact_test",)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.CHECKPOINT_PERIOD = 2000
    cfg.SOLVER.STEPS = (21000, 50000)
    cfg.SOLVER.MAX_ITER = 200000
    cfg.SOLVER.BASE_LR = 0.001
    cfg.TEST.EVAL_PERIOD = 2000
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
    cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True
    return cfg

In [4]:
classes = ["helmet"]
MetadataCatalog.get("nflimpact").set(thing_classes=classes)
cfg = new_model_cfg()
cfg.MODEL.WEIGHTS = "../input/nflimpactdetectron/model_v2_0071999.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [5]:
import pandas as pd
import numpy as np
from typing import List
import os
import matplotlib.pyplot as plt
import numpy
import sklearn
import itertools
from scipy.spatial.distance import euclidean
import pickle
import math
import sys
from collections import defaultdict
import keras
import tensorflow as tf


import cv2
from PIL import Image
import matplotlib.patches as patches
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
import io

In [6]:
import torch, torchvision

In [7]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from typing import List

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [8]:
import cv2
from detectron2.data import MetadataCatalog
from detectron2.data.detection_utils import read_image
from detectron2.structures import Instances
from detectron2.utils.visualizer import Visualizer

# Sensors 

In [9]:

def getStart(track2: pd.DataFrame) -> int:
    ''' Returns starting index to synchronise sensors with video '''

    track2[track2["event"] == "ball_snap"]["time"].unique()
    sensortick = np.where(track2["time"].unique() == track2[track2["event"] == "ball_snap"]["time"].unique()[0])[0][0]    

    return int(sensortick - 10/6)

def reduceCurrent(track2: pd.DataFrame,start: int) -> pd.DataFrame:
    ''' Returns sliced sensor DataFrame '''

    videotracks = track2["time"].unique()[start:]

    return track2[track2['time'].isin(videotracks)]

def getVideoSample(data: pd.DataFrame,gameKey: int,playID: int) -> List[any]:
    ''' For a given DataFrame, gameKey and playID, returns formatted values '''
    x = []
    game = data[data["gameKey"] == gameKey][data["playID"] == playID]
    start = getStart(game)
    current = reduceCurrent(game,start)    
    current = current.drop(['gameKey','playID', 'event'], axis=1)
    times = current["time"].unique()
    for i in range(len(times)):
        try:
            f1 = current[current["time"] == times[i]]
            f2 = current[current["time"] == times[i+1]]
            f3 = current[current["time"] == times[i+2]]  
            f4 = pd.concat([f1,f2,f3], ignore_index=True)  
            f4 = f4.drop(['time'], axis=1)          
            f4 = f4.to_numpy()
            x.append(f4)
        except:
            pass
    return x

def reshapeX(X: List[np.array], l:int = 66) -> np.array:
    ''' List[np.array] to np.array(np.array) '''

    N_X = len(X)
    N_X_Each = l
    N_Columns = len(X[0][0])

    Reshaped_X = np.zeros((N_X, N_X_Each, N_Columns))


    for i in range(N_X):

        if len(X[i]) != 66: ## Needs padding here
            Row_To_Pad = X[i][0]
            N_Rows_To_Pad = N_X_Each- len(X[i])

            for j in range(N_X_Each):
                if j < N_Rows_To_Pad:
                    Reshaped_X[i,j,:] = Row_To_Pad

                else:
                    Reshaped_X[i,j,:] = X[i][j-N_Rows_To_Pad]
        else: 
            for j in range(N_X_Each):
                Reshaped_X[i,j,:] = X[i][j]

    Reshaped_X = Reshaped_X.reshape(Reshaped_X.shape[0], Reshaped_X.shape[1]*Reshaped_X.shape[2])

    return Reshaped_X

def get_sensor_weights() -> any:
    ''' Returns classifier with loaded weights'''
    path = "../input/sendor-model/knn.pkl"
    with open(path, 'rb') as weights:
        clf = pickle.load(weights)

    return clf

def get_frames_from_sensor(predictions: List[any]) -> np.array:
    ''' Returns all the frames where a collision might take place'''

    indexes = np.where(predictions==1)
    indexes = [(x+1)*6 for x in indexes]

    return indexes[0]

def Get_Frames_From_Indexes(Indexes):
    Frames = []
    for i in range(len(Indexes)):
        if i != (len(Indexes)-1) and Indexes[i+1]==6+Indexes[i]:
            to_add = list(np.arange(Indexes[i], Indexes[i+1], 1))
            Frames = Frames + to_add
        elif i == (len(Indexes)-1):
            to_add = list(np.arange(Indexes[i], Indexes[i]+6, 1))
            Frames = Frames + to_add
        else:
            Frames = Frames + [Indexes[i]]
    return Frames

In [10]:
def get_sensor_frames(gameKey: int, playID: int, dataset: pd.DataFrame) -> np.array:
    '''For a given gameKey, playID and dataset returns the possible impact frames'''
    gameKey = gameKey
    playID = playID
    x = getVideoSample(dataset, gameKey, playID)
    x_test = reshapeX(x)
    clf = get_sensor_weights()
    y_pred = clf.predict(x_test)
    frames = get_frames_from_sensor(y_pred)
    frames = Get_Frames_From_Indexes(frames)

    return frames

# Detectron 2

In [11]:
def get_pred_boxes(instance: Instances) -> np.array:
    pred_boxes = instance.pred_boxes
    pred_boxes_array = pred_boxes.tensor.numpy()

    return pred_boxes_array

def image_detection(image: np.array) -> np.array:

    #predictor = get_predictor()
    model_output = predictor(image)    
    model_output = model_output["instances"].to("cpu")
    filter_mask = model_output.scores > 0.8
    ni = Instances(model_output.image_size,  pred_boxes = model_output.pred_boxes[filter_mask])
    pred_boxes = get_pred_boxes(ni)

    return pred_boxes

def create_pairs(pred_boxes: np.array) -> List[any]:
    pair_wise = list(itertools.combinations(pred_boxes, 2))
    return pair_wise

def euclidean_distances(pair:np.array, x: int = 1) -> (float,float,float):
    ''' Returns euclidean distance between centroids, left to right points and right to left points '''

    pairA = pair[0]
    pairB = pair[1]
    u,v = centroids(pairA,pairB)
    centroid_euclidean = euclidean(u,v)
    if x:
        left_a, right_a = x_points(pairA)
        left_b, right_b = x_points(pairB)
    else:
        left_a, right_a = y_points(pairA)
        left_b, right_b = y_points(pairB)
    a_b_euclidean = euclidean(left_a,right_b)
    b_a_euclidean = euclidean(left_b,right_a)

    return  centroid_euclidean, a_b_euclidean , b_a_euclidean

def y_points(bounding_box:np.array) -> (np.array, np.array):
    ''' Returns the top and bottom points given a bounding box '''

    x_mean = (bounding_box[0] + bounding_box[2])/2
    up = np.array([ x_mean, bounding_box[1]])
    down = np.array([ x_mean, bounding_box[3]])
    return up, down

def x_points(bounding_box:np.array) -> (np.array, np.array):
    ''' Returns the left and right points given a bounding box '''

    y_mean = (bounding_box[1] + bounding_box[3])/2
    left = np.array([bounding_box[0], y_mean])
    right = np.array([bounding_box[2], y_mean])
    return left, right

def centroids(pairA: np.array, pairB: np.array) -> (np.array, np.array):
    ''' Returns centroids given two points '''

    x_mean_A = (pairA[0] + pairA[2])/2
    y_mean_A = (pairA[1] + pairA[3])/2

    x_mean_B = (pairB[0] + pairB[2])/2
    y_mean_B = (pairB[1] + pairB[3])/2

    u = [x_mean_A, y_mean_A]
    v = [x_mean_B, y_mean_B]

    return u,v

def getMin(center_distance: np.array, distance_1: np.array, distance_2: np.array,view : int = 1,k: int = 3) -> List:
    ''' Returns the k most likely collision indexes '''
    threshold1 = 25 #13.5
    threshold2 = 25
    threshold3 = 13.5
    possible_collisions = []
    for i in range(k):
        min_center_index = np.argsort(center_distance)[i]
        min_distance_1_index = np.argsort(distance_1)[i]
        min_distance_2_index = np.argsort(distance_2)[i]
        distances = [center_distance[min_center_index], distance_1[min_distance_1_index], distance_2[min_distance_2_index]]
        distances_index = [min_center_index, min_distance_1_index, min_distance_2_index]
        collision = distances_index[np.argsort(distances)[0]]
        
        if view == 2:
            if distances[np.argsort(distances)[0]] < threshold3:
                possible_collisions.append(collision)
        else:
            if distances[np.argsort(distances)[0]] < threshold2:
                possible_collisions.append(collision)
                
    return possible_collisions

def get_distances(pair_wise: List[any], typ : int = 1) -> List[any]:
    if typ:
        center_distance = []
        distance_1 = []
        distance_2 = []

        for pair in pair_wise:
            center, d1, d2 = euclidean_distances(pair,1)
            center_distance.append(center)
            distance_1.append(d1)
            distance_2.append(d2)

        collision_pairs = getMin(center_distance, distance_1, distance_2,2) 
        return collision_pairs
    else:
        distances= []    
        for i in range(2):
            center_distance = []
            distance_1 = []
            distance_2 = []

            for pair in pair_wise:
                center, d1, d2 = euclidean_distances(pair,i)
                center_distance.append(center)
                distance_1.append(d1)
                distance_2.append(d2)

            collision_pairs = getMin(center_distance, distance_1, distance_2,0)    
            distances.append(collision_pairs)    
    
        x_list = distances[0]
        y_list = distances[1]
        
        return list(set(x_list) & set(y_list))

def From_Detectron2_To_NFL(BB_Detectron: List[any]) -> List[any]:
    x1 = BB_Detectron[0]
    x2 = BB_Detectron[2]
    y1 = BB_Detectron[1]
    y2 = BB_Detectron[3]

    width = x2-x1
    height = y2 - y1
    left = x1
    top = y1

    BB_NFL = [left, width, top, height]

    return BB_NFL 


In [12]:
def from_Video_to_Prediction(video: str, frames: List[any], line : int = 1) -> pd.DataFrame:
    ''' Line 1 for Sideline '''

    predictions = {}
    vidcap = cv2.VideoCapture(video)
    success,image = vidcap.read()
    counter = 1
    temp_dics = []
    while success:
        if counter in frames:
            bounding_boxes = image_detection(image)
            pair_wise = create_pairs(bounding_boxes)
            collision_test = get_distances(pair_wise,line)        
            temp_dic = {"video_name" : video, "frame": counter}
            for value in collision_test:
                pair = pair_wise[value]
                element_1 = From_Detectron2_To_NFL(pair[0])
                element_2 = From_Detectron2_To_NFL(pair[1])
                temp_dic2 = temp_dic.copy()
                temp_dic2["P1left"] = element_1[0]
                temp_dic2["P1width"] = element_1[1]
                temp_dic2["P1top"] = element_1[2]
                temp_dic2["P1height"] = element_1[3]
                temp_dic2["P2left"] = element_2[0]
                temp_dic2["P2width"] = element_2[1]
                temp_dic2["P2top"] = element_2[2]
                temp_dic2["P2height"] = element_2[3]
                temp_dics.append(temp_dic2)

        counter += 1
        success,image = vidcap.read()
    df = pd.DataFrame(temp_dics)
    return df

# Getting images

In [13]:
def reformat_bounding_box_coordinates(left: int, width: int, top:int, height:int) -> tuple:
    """
    Clock wise format of the 2d points. Starting from the
    top left.

    x-----x
    |     |
    x-----x
    """
    x_coordinates = [left, left + width, left+width, left]
    y_coordinates = [top, top, top + height, top + height]

    return x_coordinates, y_coordinates

def get_x1_y1_x2_y2_format(bb:list)-> List[tuple]:
    x1 = bb[0]
    y1 = bb[2]
    x2 = x1 + bb[1]
    y2 = y1 + bb[3]
    
    start = (x1, y1)
    end = (x2, y2)
    
    return [start, end]

def minimum_bounding_box(bb1: list, bb2: list, extra_context:int = 0) -> list:
    """
    Given two bounding boxes where the format is as follows:
        bb = [left, widht, top, height]
    Computes the minimum bounding box that contains both bounding boxes.
    """
    
    # Clock Wise Format
    bb1_coordinates = reformat_bounding_box_coordinates(*bb1)
    bb2_coordinates = reformat_bounding_box_coordinates(*bb2)

    # Left and right x coordinates
    bb1_x_coordinates = set(bb1_coordinates[0])
    bb2_x_coordinates = set(bb2_coordinates[0])

    x_coordinates = list(bb1_x_coordinates) + list(bb2_x_coordinates)

    min_width = min(x_coordinates)
    max_width = max(x_coordinates)

    left = min_width
    width = max_width - min_width

    # Top and bottom y coordinates
    bb1_y_coordinates = set(bb1_coordinates[1])
    bb2_y_coordinates = set(bb2_coordinates[1])

    y_coordinates = list(bb1_y_coordinates) + list(bb2_y_coordinates)

    min_height = min(y_coordinates)
    max_height = max(y_coordinates)

    top = min_height
    height = max_height - min_height

    return  max(left - extra_context, 0), width + 2*extra_context, max(top - extra_context, 0), height + 2*extra_context



def center_image_and_pad(image:np.array, size:tuple):
    ht, wd, cc = image.shape
    ww, hh = size

    wd = min(ww, wd)
    ht = min(hh, ht)
    
    # Crop in case is bigger
    image = image[:ht, :wd]
    
    color = (0,0,0)
    result = np.full((hh,ww,cc), color, dtype=np.uint8)

    # compute center offset
    xx = (ww - wd) // 2
    yy = (hh - ht) // 2

    # copy img image into center of result image
    result[yy:yy+ht, xx:xx+wd] = image
    
    return result



def get_specific_frame_from_video(video_path: str, frame:int) -> np.array:
    """
    Given the path of a video extract the specific nth frame.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = cap.get(7)
    cap.set(1, frame)
    ret, frame = cap.read()
    
    return frame



def add_bounding_boxes_to_frame(frame, bounding_boxes:list) -> np.array:
    for bb in bounding_boxes:
        start, end = get_x1_y1_x2_y2_format(bb)
        frame = cv2.rectangle(frame, start, end, (255, 0, 0), 1)
        
    return frame



def get_min_bounding_box_image(video_name: str, frame: int, bb1:list, bb2:list, extra_context=0) -> np.array:
    frame = get_specific_frame_from_video(video_name, frame)   
    min_bb = minimum_bounding_box(bb1, bb2, extra_context=extra_context)
    
    # Get x1, y1, x2, y2
    bb_x1 = min_bb[0]
    bb_y1 = min_bb[2]
    bb_x2 = bb_x1 + min_bb[1]
    bb_y2 = bb_y1 + min_bb[3]
    
    # Slice the part we want
    sliced_frame = frame[bb_y1:bb_y2, bb_x1:bb_x2]
    
    # Pad the image 
    sliced_frame = center_image_and_pad(sliced_frame, (80,80))
    
    return sliced_frame



def get_min_bounding_box_image_with_bbs(video_name: str, frame: int, bb1:list, bb2:list, extra_context=0) -> np.array:
    frame = get_specific_frame_from_video(video_name, frame)   
    min_bb = minimum_bounding_box(bb1, bb2, extra_context=extra_context)
    
    # Append the bounding boxes
    frame = add_bounding_boxes_to_frame(frame, [bb1, bb2])
    
    # Transform the format
    start, end = get_x1_y1_x2_y2_format(min_bb)
    x1, y1 = start
    x2, y2 = end
    
    # Slice the part we want
    sliced_frame = frame[y1:y2, x1:x2]
    
    # Pad the image
    sliced_frame = center_image_and_pad(sliced_frame, (80,80))
    
    return sliced_frame


In [14]:
def get_list_images(df: pd.DataFrame) -> List[any]:
    image_list = np.empty(shape = (len(df.index), 80,80,3),dtype=np.float32)
    count = 0
    for index, row in df.iterrows():
        video_name = row.video_name
        frame = row.frame
        bb1 = [int(row["P1left"]), int(row["P1width"]), int(row["P1top"]), int(row["P1height"])]
        bb2 = [int(row["P2left"]), int(row["P2width"]), int(row["P2top"]), int(row["P2height"])]

        img = get_min_bounding_box_image_with_bbs(video_name, frame, bb1, bb2, extra_context=10)
        img = tf.keras.applications.resnet.preprocess_input(img)
        image_list[count] = img
        count+= 1
    return image_list

In [15]:
def load_cnn() -> any:
    model = keras.models.load_model('../input/sideline-resnet-model-bb-1/Sideline_BB_model (1).h5')
    return model

In [16]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    df2['centerx1'] = df2.apply(lambda row: row.P1left + row.P1width, axis=1)
    df2['centery1'] = df2.apply(lambda row: row.P1top + row.P1height, axis=1)
    df2['centerx2'] = df2.apply(lambda row: row.P2left + row.P2width, axis=1)
    df2['centery2'] = df2.apply(lambda row: row.P2top + row.P2height, axis=1)  
    to_drop = []
    for i in range(len(df2)):
        current = df2.iloc[i]
        current_frame = current["frame"]
        for j in range(i+1,len(df2)):
            second = df2.iloc[j]
            second_frame = second["frame"]
            if (second_frame - current_frame < 2):
                u = [current["centerx1"], current["centery1"]]
                v = [second["centerx1"], second["centery1"]]        
                u1 = [current["centerx2"], current["centery2"]]
                v1 = [second["centerx2"], second["centery2"]]
                w = [(u[0] + u1[0])/2,(u[1] + u1[1])/2]
                w1 = [(v[0] + v1[0])/2,(v[0] + v1[1])/2]
                distance = euclidean(u,v)
                distance1 = euclidean(u1,v1)
                distance2 = euclidean(u,v1)
                distance3 = euclidean(u1,v)
                distance4 = euclidean(w,w1)
                if (distance < 2 or distance1 < 2 or distance2 < 2 or distance3 < 2 or distance4 < 2):
                    to_drop.append(j)

    df = df.drop(to_drop)
    df = df.reset_index()
    return df

In [17]:
def reduce_predictions(values: np.array, threshold: float) -> np.array:
    values[np.where(values >threshold)] = 1
    values[np.where(values <= threshold)] = 0
    return values

In [18]:
def get_final_dataframe(collision_frames: pd.DataFrame, gameKey: int, playID: int, video: str, view: int) -> pd.DataFrame:
    final_format = []
    for index, row in collision_frames.iterrows():
        if view:
            v = "Sideline"
        else:
            v = "Endzone"
        temp_dic = {"gameKey": gameKey, "playID": playID, "view": v, "video": video, "frame": row.frame}
        bb1 = [int(row["P1left"]), int(row["P1width"]), int(row["P1top"]), int(row["P1height"])]
        bb2 = [int(row["P2left"]), int(row["P2width"]), int(row["P2top"]), int(row["P2height"])]
        min_bounding_box = minimum_bounding_box(bb1,bb2)
        temp_dic["left"] = min_bounding_box[0]
        temp_dic["width"] = min_bounding_box[1]
        temp_dic["top"] = min_bounding_box[2]
        temp_dic["height"] = min_bounding_box[3]
        final_format.append(temp_dic)
    df = pd.DataFrame(final_format)
    return df

In [19]:
Video_Names = os.listdir('../input/nfl-impact-detection/test')
Video_Paths = []
gameKey = []
playID = []
view = []

print(type(Video_Names[0].split()))


for i in range(len(Video_Names)):
    gameKey.append(int(Video_Names[i].split('_')[0]))
    playID.append(int(Video_Names[i].split('_')[1]))
    Video_Paths.append('../input/nfl-impact-detection/test/' + Video_Names[i])
    
print(playID)
print(Video_Paths)
print(gameKey)

<class 'list'>
[718, 718, 109, 2798, 109, 2798]
['../input/nfl-impact-detection/test/57906_000718_Sideline.mp4', '../input/nfl-impact-detection/test/57906_000718_Endzone.mp4', '../input/nfl-impact-detection/test/57995_000109_Endzone.mp4', '../input/nfl-impact-detection/test/58102_002798_Sideline.mp4', '../input/nfl-impact-detection/test/57995_000109_Sideline.mp4', '../input/nfl-impact-detection/test/58102_002798_Endzone.mp4']
[57906, 57906, 57995, 58102, 57995, 58102]


In [20]:
import time

df = pd.read_csv("../input/nfl-impact-detection/test_player_tracking.csv")
df['player'] = df['player'].apply(lambda x: 1 if 'H' in x else 0)
model = load_cnn()

submission_df = pd.DataFrame()

for i, vid in enumerate(Video_Names):
    if("Sideline" in vid):
        print("Checking video", vid)
        game = gameKey[i]
        play = playID[i]
        videopath = Video_Paths[i]
        print("Getting sensor info!")
        sensors = get_sensor_frames(game,play,df)
        video1 = from_Video_to_Prediction(videopath, sensors, 1)
        trimmed_video = remove_duplicates(video1)
        image_list = get_list_images(trimmed_video)
        print("CNN TIME!")
        predictions = model.predict(image_list)
        predictions = reduce_predictions(predictions, 0.55)
        positions = np.where(predictions == 1)[0].tolist()
        collision_frames = trimmed_video.iloc[positions]
        final_df = get_final_dataframe(collision_frames,game, play, vid,1)
        print("DATAFRAME", final_df)
        submission_df = submission_df.append(final_df, ignore_index = True)
        print("Current state", submission_df)
    else:
        print("Checking video", vid)
        game = gameKey[i]
        play = playID[i]
        videopath = Video_Paths[i]
        print("Getting sensor info!")
        sensors = get_sensor_frames(game,play,df)
        video1 = from_Video_to_Prediction(videopath, sensors, 0)
        trimmed_video = remove_duplicates(video1)
        image_list = get_list_images(trimmed_video)
        print("CNN TIME!")
        final_df = get_final_dataframe(trimmed_video,game, play, vid, 0)
        print("DATAFRAME", final_df)
        submission_df = submission_df.append(final_df, ignore_index = True)
        print("Current state", submission_df)


Checking video 57906_000718_Sideline.mp4
Getting sensor info!


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  filter_inds = filter_mask.nonzero()


CNN TIME!
DATAFRAME    gameKey  playID      view                      video  frame  left  width  \
0    57906     718  Sideline  57906_000718_Sideline.mp4     33   819     19   
1    57906     718  Sideline  57906_000718_Sideline.mp4     39   790     20   
2    57906     718  Sideline  57906_000718_Sideline.mp4     40   817     13   
3    57906     718  Sideline  57906_000718_Sideline.mp4     41   855     16   
4    57906     718  Sideline  57906_000718_Sideline.mp4     44   858     16   
5    57906     718  Sideline  57906_000718_Sideline.mp4     45   792     17   
6    57906     718  Sideline  57906_000718_Sideline.mp4    111   799     17   

   top  height  
0  376      21  
1  357      14  
2  379      10  
3  399      15  
4  397      17  
5  356      13  
6  343      10  
Current state    gameKey  playID      view                      video  frame  left  width  \
0    57906     718  Sideline  57906_000718_Sideline.mp4     33   819     19   
1    57906     718  Sideline  57906_000

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CNN TIME!
DATAFRAME     gameKey  playID     view                     video  frame  left  width  \
0     57906     718  Endzone  57906_000718_Endzone.mp4     24   799     40   
1     57906     718  Endzone  57906_000718_Endzone.mp4     29   609     42   
2     57906     718  Endzone  57906_000718_Endzone.mp4     30   804     25   
3     57906     718  Endzone  57906_000718_Endzone.mp4     30   628     33   
4     57906     718  Endzone  57906_000718_Endzone.mp4     31   625     36   
5     57906     718  Endzone  57906_000718_Endzone.mp4     35   620     38   
6     57906     718  Endzone  57906_000718_Endzone.mp4     36   621     37   
7     57906     718  Endzone  57906_000718_Endzone.mp4     37   621     37   
8     57906     718  Endzone  57906_000718_Endzone.mp4     39   616     25   
9     57906     718  Endzone  57906_000718_Endzone.mp4     45   610     30   
10    57906     718  Endzone  57906_000718_Endzone.mp4     45   367     21   
11    57906     718  Endzone  57906_000718_E

...

CNN TIME!
DATAFRAME Empty DataFrame
Columns: []
Index: []
Current state     gameKey  playID      view                      video  frame  left  width  \
0     57906     718  Sideline  57906_000718_Sideline.mp4     33   819     19   
1     57906     718  Sideline  57906_000718_Sideline.mp4     39   790     20   
2     57906     718  Sideline  57906_000718_Sideline.mp4     40   817     13   
3     57906     718  Sideline  57906_000718_Sideline.mp4     41   855     16   
4     57906     718  Sideline  57906_000718_Sideline.mp4     44   858     16   
..      ...     ...       ...                        ...    ...   ...    ...   
75    57995     109   Endzone   57995_000109_Endzone.mp4    133   874     31   
76    57995     109   Endzone   57995_000109_Endzone.mp4    133   439     37   
77    57995     109   Endzone   57995_000109_Endzone.mp4    136   443     39   
78    57995     109   Endzone   57995_000109_Endzone.mp4    136  1046     37   
79    57995     109   Endzone   57995_000109_End

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CNN TIME!
DATAFRAME    gameKey  playID      view                      video  frame  left  width  \
0    57995     109  Sideline  57995_000109_Sideline.mp4     41   489     24   
1    57995     109  Sideline  57995_000109_Sideline.mp4     47   495     19   
2    57995     109  Sideline  57995_000109_Sideline.mp4     50   493     23   
3    57995     109  Sideline  57995_000109_Sideline.mp4     51   486     25   
4    57995     109  Sideline  57995_000109_Sideline.mp4     53   497     18   
5    57995     109  Sideline  57995_000109_Sideline.mp4     55   483     20   
6    57995     109  Sideline  57995_000109_Sideline.mp4     56   493     25   
7    57995     109  Sideline  57995_000109_Sideline.mp4     62   473     27   
8    57995     109  Sideline  57995_000109_Sideline.mp4    106   432     12   
9    57995     109  Sideline  57995_000109_Sideline.mp4    135   455     22   

   top  height  
0  325      17  
1  319      21  
2  318      16  
3  218      17  
4  320      20  
5  211  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CNN TIME!
DATAFRAME     gameKey  playID     view                     video  frame  left  width  \
0     58102    2798  Endzone  58102_002798_Endzone.mp4     36   487     30   
1     58102    2798  Endzone  58102_002798_Endzone.mp4     40   484     36   
2     58102    2798  Endzone  58102_002798_Endzone.mp4     41   492     45   
3     58102    2798  Endzone  58102_002798_Endzone.mp4     48   839     49   
4     58102    2798  Endzone  58102_002798_Endzone.mp4    127   444     47   
5     58102    2798  Endzone  58102_002798_Endzone.mp4    131   447     48   
6     58102    2798  Endzone  58102_002798_Endzone.mp4    132   930     46   
7     58102    2798  Endzone  58102_002798_Endzone.mp4    133   447     51   
8     58102    2798  Endzone  58102_002798_Endzone.mp4    134   450     51   
9     58102    2798  Endzone  58102_002798_Endzone.mp4    134   935     50   
10    58102    2798  Endzone  58102_002798_Endzone.mp4    135   452     52   
11    58102    2798  Endzone  58102_002798_E

In [21]:
submission_df

,gameKey,playID,view,video,frame,left,width,top,height
0,57906,718,Sideline,57906_000718_Sideline.mp4,33,819,19,376,21
1,57906,718,Sideline,57906_000718_Sideline.mp4,39,790,20,357,14
2,57906,718,Sideline,57906_000718_Sideline.mp4,40,817,13,379,10
3,57906,718,Sideline,57906_000718_Sideline.mp4,41,855,16,399,15
4,57906,718,Sideline,57906_000718_Sideline.mp4,44,858,16,397,17
...,...,...,...,...,...,...,...,...,...
103,58102,2798,Endzone,58102_002798_Endzone.mp4,158,941,47,112,41
104,58102,2798,Endzone,58102_002798_Endzone.mp4,160,938,46,114,37
105,58102,2798,Endzone,58102_002798_Endzone.mp4,161,934,47,115,35
106,58102,2798,Endzone,58102_002798_Endzone.mp4,162,933,44,117,33


In [22]:
submission_df = submission_df.reset_index()

In [23]:
image_list

array([[[[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],

        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],

        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ]],

        ...,

        [[-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         [-103.939, -116.779, -123.68 ],
         ...,
         [-103.939, -116.779, -123.68 ],
         [-103.939, -1

In [24]:
submission_df = submission_df.drop(columns=["index"])

In [25]:
df = submission_df

In [26]:
submission_df

,gameKey,playID,view,video,frame,left,width,top,height
0,57906,718,Sideline,57906_000718_Sideline.mp4,33,819,19,376,21
1,57906,718,Sideline,57906_000718_Sideline.mp4,39,790,20,357,14
2,57906,718,Sideline,57906_000718_Sideline.mp4,40,817,13,379,10
3,57906,718,Sideline,57906_000718_Sideline.mp4,41,855,16,399,15
4,57906,718,Sideline,57906_000718_Sideline.mp4,44,858,16,397,17
...,...,...,...,...,...,...,...,...,...
103,58102,2798,Endzone,58102_002798_Endzone.mp4,158,941,47,112,41
104,58102,2798,Endzone,58102_002798_Endzone.mp4,160,938,46,114,37
105,58102,2798,Endzone,58102_002798_Endzone.mp4,161,934,47,115,35
106,58102,2798,Endzone,58102_002798_Endzone.mp4,162,933,44,117,33


In [27]:
df.keys()

Index(['gameKey', 'playID', 'view', 'video', 'frame', 'left', 'width', 'top',
       'height'],
      dtype='object')

In [28]:
import nflimpact
env = nflimpact.make_env()
env.predict(df) # df is a pandas dataframe of your entire submission file